In [1]:
from collections import Counter, defaultdict

import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

tqdm.pandas()
import itertools

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import nltk
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import spacy
import umap.umap_ as umap
from nltk.corpus import stopwords
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [2]:
# Run this in your environment in order load the pre-trained model
# python -m spacy download en_core_web_sm


# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

## Load Data and obtain Embeddings

In [3]:
dataset_name = "banking" # 'stackoverflow' 'banking' 'clinc'
model = SentenceTransformer("all-MiniLM-L6-v2")

In [4]:
# Ensure stopwords are downloaded for lexical analysis
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/padeck/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_train = pd.read_csv(f"../data/{dataset_name}/train.tsv",sep="\t")
df_eval = pd.read_csv(f"../data/{dataset_name}/dev.tsv",sep="\t")
df_test = pd.read_csv(f"../data/{dataset_name}/test.tsv",sep="\t")
df = pd.concat([df_train, df_eval, df_test])
del df_train
del df_eval
del df_test

In [ ]:
def calc_embeddings(text):
    return model.encode(text,normalize_embeddings=True)

df['text_embds'] = df['text'].progress_apply(calc_embeddings)

 42%|████▏     | 5487/13083 [01:15<01:37, 78.10it/s]

## UMAP Projection

In [ ]:
# Stack embeddings into a 2D array
embedding_matrix = np.vstack(df['text_embds'].values)

# UMAP dimensionality reduction
reducer = umap.UMAP(n_neighbors=30, min_dist=0.3, metric='cosine', random_state=42)
embedding_2d = reducer.fit_transform(embedding_matrix)

# Add 2D projection to original dataframe
df['x'] = embedding_2d[:, 0]
df['y'] = embedding_2d[:, 1]

# Compute label-wise centroids in original embedding space
label_centroids = df.groupby('label')['text_embds'].apply(
    lambda x: np.mean(np.vstack(x.values), axis=0)
)

# Transform centroids to 2D using UMAP
centroid_embeddings = np.vstack(label_centroids.values)
centroid_2d = reducer.transform(centroid_embeddings)
centroids_df = pd.DataFrame(centroid_2d, columns=['x', 'y'])
centroids_df['label'] = label_centroids.index

fig = go.Figure()

# Add data points grouped by label
for label in df['label'].unique():
    subset = df[df['label'] == label]
    fig.add_trace(
        go.Scatter(
            x=subset['x'],
            y=subset['y'],
            mode='markers',
            name=str(label),
            marker=dict(size=4, opacity=1.0),
            text=subset['text'],
            hoverinfo='text'
        )
    )

# Add centroids last — so they are on top
fig.add_trace(
    go.Scatter(
        x=centroids_df['x'],
        y=centroids_df['y'],
        mode='markers+text',
        name='Centroids',
        marker=dict(
            color='black',
            size=10,
            symbol='x',
            line=dict(width=2, color='white')
        ),
        text=centroids_df['label'],
        textposition='top center',
        textfont=dict(size=14, color='black'),
        hoverinfo='text'
    )
)

# Layout
fig.update_layout(
    title=f'UMAP Projection for the {dataset_name} dataset',
    width=900,
    height=700,
    showlegend=True
)

fig.show()


## Lexical Analysis

In [ ]:
# Function to extract nouns and verbs
def extract_nouns_verbs(text):
    doc = nlp(text)
    nouns = [token.lemma_.lower() for token in doc if token.pos_ == 'NOUN' and not token.is_stop and token.is_alpha]
    verbs = [token.lemma_.lower() for token in doc if token.pos_ == 'VERB' and not token.is_stop and token.is_alpha]
    return nouns, verbs

# Initialize result dictionary
label_stats = {}

# Enable tqdm for the outer loop over labels
for label, group in tqdm(df.groupby('label'), desc="Processing Labels"):
    all_nouns = []
    all_verbs = []

    # tqdm for inner loop over texts in each label
    for text in tqdm(group['text'], desc=f"Texts in '{label}'", leave=False):
        nouns, verbs = extract_nouns_verbs(text)
        all_nouns.extend(nouns)
        all_verbs.extend(verbs)

    noun_counts = Counter(all_nouns).most_common(3)
    verb_counts = Counter(all_verbs).most_common(3)

    label_stats[label] = {
        'top_nouns': [word for word, _ in noun_counts],
        'top_verbs': [word for word, _ in verb_counts]
    }

# Convert results to DataFrame
result_df = pd.DataFrame.from_dict(label_stats, orient='index').reset_index()
result_df.columns = ['label', 'top_nouns', 'top_verbs']
result_df

## Clustering Metrics

### Intra-Class Similarities

In [ ]:
# Convert embeddings into a matrix
embedding_matrix = np.vstack(df["text_embds"].values)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(embedding_matrix)

# Store in DataFrame for better readability
similarity_df = pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)

In [ ]:
intra_label_stats = defaultdict(lambda: {"Avg Similarity": 0, "Variance": 0})

labels = df['label'].unique()

for label in labels:
    indices = df[df["label"] == label].index  # Get indices of this intent
    sims = similarity_df.loc[indices, indices].values  # Extract similarity scores
    np.fill_diagonal(sims, np.nan)  # Ignore self-similarity (diagonal values)
    
    avg_sim = np.nanmean(sims)  # Compute mean similarity
    variance_sim = np.nanvar(sims)  # Compute variance
    
    intra_label_stats[label]["Avg Similarity"] = avg_sim
    intra_label_stats[label]["Variance"] = variance_sim

# Convert to DataFrame
intra_df = pd.DataFrame.from_dict(intra_label_stats, orient="index").reset_index()
intra_df.columns = ["Intent", "Avg Intra Similarity", "Variance"]

# Sort by average similarity
intra_df = intra_df.sort_values(by="Avg Intra Similarity", ascending=False)
intra_df

In [ ]:
# Averaging over all avg. intra class similarities for an overall metric
intra_df['Avg Intra Similarity'].mean()

### Inter-Category Similarities

In [ ]:
# List of all unique intent labels
intent_pairs = list(itertools.combinations(labels, 2))  # All possible intent pairs

# Function to compute inter-intent similarity
def compute_inter_intent_similarity(df, intent_a, intent_b):
    # Extract embeddings for each intent
    emb_a = np.stack(df[df["label"] == intent_a]["text_embds"].values)
    emb_b = np.stack(df[df["label"] == intent_b]["text_embds"].values)
    
    # Compute cosine similarity between all utterances
    return np.mean(cosine_similarity(emb_a, emb_b))

# Compute similarity for all intent pairs and store in a dict
inter_intent_similarities = {
    (intent_a, intent_b): compute_inter_intent_similarity(df, intent_a, intent_b)
    for intent_a, intent_b in intent_pairs
}

# Create a symmetric DataFrame for the similarity matrix
intent_sim_matrix = pd.DataFrame(index=labels, columns=labels, dtype=float)

# Populate the upper triangle of the matrix (no need to fill the lower triangle)
for (intent_a, intent_b), sim in inter_intent_similarities.items():
    intent_sim_matrix.loc[intent_a, intent_b] = sim
    intent_sim_matrix.loc[intent_b, intent_a] = sim  # Symmetric

# Optionally, fill diagonal values with NaN (since comparing same intents is not necessary)
np.fill_diagonal(intent_sim_matrix.values, np.nan)

# Display the resulting similarity matrix
intent_sim_matrix

In [ ]:
# Assuming centroid_sim_df is already defined as a similarity matrix
# Mask the diagonal (self-similarity) by setting it to NaN
np.fill_diagonal(intent_sim_matrix.values, np.nan)

# Replace values that are 1 or 0 with NaN (self-similarity or exact dissimilarity)
intent_sim_df = intent_sim_matrix.replace({1: np.nan, 0: np.nan})

# Keep only the upper triangle to remove redundant pairs
centroid_sim_df = intent_sim_df.where(np.triu(np.ones(intent_sim_df.shape), k=1).astype(bool))

# Flatten the matrix, sort values, and get the top 100 highest similarity pairs
most_similar = centroid_sim_df.stack().nlargest(100)


# Display top 5 highest similarity pairs
print("Most Similar Pairs:")
for (pair, similarity) in most_similar.items():
    print(f"The pair '{pair[0]}' and '{pair[1]}' has a cosine similarity of {similarity:.4f}")


In [ ]:
# Function to find the most similar utterance-pair from the most similar cluster pair
def find_most_similar_utterance_pair(df, top_clusters):
    most_similar_utterances = []
    
    # Loop through the most similar cluster pairs
    for cluster_1, cluster_2 in top_clusters:
        # Extract the data for both clusters
        cluster_1_entries = df[df['label'] == cluster_1]
        cluster_2_entries = df[df['label'] == cluster_2]
        
        # Get the embeddings for the utterances in these clusters
        cluster_1_embeddings = np.array(cluster_1_entries['text_embds'].tolist())
        cluster_2_embeddings = np.array(cluster_2_entries['text_embds'].tolist())
        
        # Compute cosine similarities between the utterances from the two clusters
        similarity_matrix = cosine_similarity(cluster_1_embeddings, cluster_2_embeddings)
        
        # Find the index of the most similar utterance-pair
        idx_1, idx_2 = np.unravel_index(np.argmax(similarity_matrix), similarity_matrix.shape)
        
        # Get the most similar utterance-pair and its similarity score
        similarity_score = similarity_matrix[idx_1, idx_2]
        
        # Add to the results list with cluster labels
        most_similar_utterances.append({
            'cluster_1_label': cluster_1,  # Label of cluster 1
            'cluster_2_label': cluster_2,  # Label of cluster 2
            'cluster_1_utterance': cluster_1_entries.iloc[idx_1]['text'],  # Original text from cluster 1
            'cluster_2_utterance': cluster_2_entries.iloc[idx_2]['text'],  # Original text from cluster 2
            'similarity': similarity_score
        })
    
    return most_similar_utterances

# Get the top similar clusters (using the `top_5_similar` from previous code)
# Assuming `top_5_similar.index` is a list of tuples: (cluster_1, cluster_2)
top_clusters = most_similar.index

# Find the most similar utterance-pair for the most similar clusters
most_similar_utterances = find_most_similar_utterance_pair(df, top_clusters)

# Display the most similar utterance-pair(s) for each cluster pair, including cluster labels
print("\nMost Similar Utterance Pair Between the Most Similar Clusters:")
for entry in most_similar_utterances:
    print(f"Cluster 1 (Label: '{entry['cluster_1_label']}') Utterance: '{entry['cluster_1_utterance']}'")
    print(f"Cluster 2 (Label: '{entry['cluster_2_label']}') Utterance: '{entry['cluster_2_utterance']}'")
    print(f"Cosine Similarity: {entry['similarity']:.4f}")
    print("-" * 50)


### DBI and Silhouette Score

In [ ]:
# Function to calculate cosine distance
def cosine_distance(v1, v2):
    return 1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# Function to compute centroids of clusters
def compute_centroids(df, label_col, emb_col):
    centroids = []
    labels = df[label_col].unique()
    for label in labels:
        cluster = np.vstack(df[df[label_col] == label][emb_col])
        centroid = np.mean(cluster, axis=0)
        centroid = normalize(centroid.reshape(1, -1))[0]
        centroids.append(centroid)
    return np.array(centroids), labels

# Function to compute Davies-Bouldin Index using cosine distance
def dbi_cosine(df, label_col, emb_col):
    centroids, labels = compute_centroids(df, label_col, emb_col)
    n_clusters = len(centroids)
    S = np.zeros(n_clusters)
    M = np.full((n_clusters, n_clusters), np.inf)

    # Compute S_i and M_ij
    for i in range(n_clusters):
        cluster_i = np.vstack(df[df[label_col] == labels[i]][emb_col])
        for x in cluster_i:
            S[i] += cosine_distance(x, centroids[i])
        S[i] /= len(cluster_i)

        for j in range(i + 1, n_clusters):
            M[i, j] = M[j, i] = cosine_distance(centroids[i], centroids[j])

    # Compute DBI
    DBI = np.mean([max((S[i] + S[j]) / M[i, j] for j in range(n_clusters) if i != j) for i in range(n_clusters)])
    return DBI

In [ ]:
# Calculate DBI
dbi_value = dbi_cosine(df, 'label', 'text_embds')
print(f"Davies-Bouldin Index (cosine): {dbi_value}")

In [ ]:
# Silhouette Score already supports cosine as a distance metric
silhouette_score(df['text_embds'].tolist(),df['label'].tolist(),metric='cosine')